# Objective
The goal of this notebook is to test various methods of plotting an image. It follows [this StackExchange post](https://stackoverflow.com/questions/30121351/fastest-pcolor-like-method-with-log-scale-support). Given particle fluxes as a function of sample times and energy bins, F(t,E), plot F as a 2D image with the E-axis log-scaled.

# Setup

In [1]:
import numpy as np
import datetime as dt
import spacepy
from spacepy import pycdf
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pymms
from pymms import mms_utils

# Create an instance of SDC object
sdc = pymms.MrMMS_SDC_API()

# Define the spacecraft. We will use the variable later when accessing the CDF files.
sc = 'mms1'
mode = 'srvy'
level = 'l2'                    # 'l2' or 'sitl'
start_date = '2015-12-06'
end_date = '2015-12-06T23:59:59'
data_root = '/Users/argall/data/mms/' # Specifying data_root="~/" does not expand the tilde yet
                                      # However, if data_root=None, then ~/data is the default

# Set attributes
sdc.sc = sc
sdc.mode = mode
sdc.start_date = start_date
sdc.end_date = end_date
sdc.data_root = data_root

# Load Data

In [2]:
# Update instrument-specific variables
des_instr = 'fpi'

if mode == 'brst':
    des_mode = mode
else:
    des_mode = 'fast'

if level == 'sitl':
    des_coords = 'dbcs'
    des_level = 'ql'
    des_optdesc = 'des'
else:
    des_coords = 'gse'
    des_level = level
    des_optdesc = 'des-moms'

# Set attributes
sdc.instr = des_instr
sdc.mode = des_mode
sdc.level = des_level
sdc.optdesc = des_optdesc

# DIS variable names
espec_vname = '_'.join((sc, 'des', 'energyspectr', 'omni', des_mode))

# Open the file
files = sdc.Download()
files = mms_utils.sort_files(files)[0]

# Read the data
des_t = []
des_espec = []
des_energy = []
for file in files:
    # Open the file
    cdf = pycdf.CDF(file)
    
    # Read timee and shift to center of interval
    #   - There must be a bug in the CDF package because the Epoch_plus_var variables
    #     are read as empty but really contain scalar values
    t = cdf[cdf[espec_vname].attrs['DEPEND_0']][:]
#    dt_minus = t.attrs['DELTA_MINUS_VAR']
#    dt_plus = t.attrs['DELTA_PLUS_VAR']
    dt_minus = 0
    dt_plus = 4.5
    t += dt.timedelta(seconds=(dt_plus - dt_minus) / 2.0)
    
    # Read the data
    des_t += list(t)
    des_espec += list(cdf[espec_vname][:])
    des_energy += list(cdf[cdf[espec_vname].attrs['DEPEND_1']][:])

    # Close the file
    cdf.close()
    print('  ' + file)

# Convert back to numpy arrays
des_t = np.array(des_t)
des_espec = np.array(des_espec)
des_energy = np.array(des_energy)

  /Users/argall/data/mms/mms1/fpi/fast/l2/des-moms/2015/12/mms1_fpi_fast_l2_des-moms_20151206000000_v3.3.0.cdf
  /Users/argall/data/mms/mms1/fpi/fast/l2/des-moms/2015/12/mms1_fpi_fast_l2_des-moms_20151206020000_v3.3.0.cdf
  /Users/argall/data/mms/mms1/fpi/fast/l2/des-moms/2015/12/mms1_fpi_fast_l2_des-moms_20151206040000_v3.3.0.cdf
  /Users/argall/data/mms/mms1/fpi/fast/l2/des-moms/2015/12/mms1_fpi_fast_l2_des-moms_20151206060000_v3.3.0.cdf
  /Users/argall/data/mms/mms1/fpi/fast/l2/des-moms/2015/12/mms1_fpi_fast_l2_des-moms_20151206080000_v3.3.0.cdf
  /Users/argall/data/mms/mms1/fpi/fast/l2/des-moms/2015/12/mms1_fpi_fast_l2_des-moms_20151206100000_v3.3.0.cdf
  /Users/argall/data/mms/mms1/fpi/fast/l2/des-moms/2015/12/mms1_fpi_fast_l2_des-moms_20151206120000_v3.3.0.cdf
  /Users/argall/data/mms/mms1/fpi/fast/l2/des-moms/2015/12/mms1_fpi_fast_l2_des-moms_20151206220000_v3.3.0.cdf


# Plot
## NonUniformImage

In [3]:
from matplotlib.image import NonUniformImage
from matplotlib import cm

t_sec = [(t - dt.datetime.strptime(start_date, '%Y-%m-%d')).total_seconds() for t in des_t]
t_sec = np.array(t_sec)

fig, axs = plt.subplots(nrows=1, ncols=1, squeeze=False, frameon=False) #figsize=(8,4))
fig.set_size_inches(8, 4)
#fig.subplots_adjust(bottom=0.07, hspace=0.3)
#fig.suptitle('NonUniformImage Class', fontsize='large')

ax = axs[0,0]
im = NonUniformImage(ax, interpolation='nearest', extent=(t_sec[0], t_sec[-1], des_energy[0,0], des_energy[0,-1]),
                     cmap='Purples')
im.set_data(t_sec, np.log(des_energy[0,:]), des_espec.transpose())
ax.images.append(im)
ax.set_title('ESpec')
ax.set_xlim(t[0], t[-1])
#ax.set_ylim(des_energy[0,0], des_energy[0,-1])

plt.show()

ValueError: Image size of 1922646225x264 pixels is too large. It must be less than 2^16 in each direction.

## PColorImage

In [ ]:
from matplotlib import PColor

fig, ax0 = plt.subplots(1, 1, squeeze=False)

c = ax.pcolor(Z)
ax0.set_title('default: no edges')

c = ax1.pcolor(Z, edgecolors='k', linewidths=4)
ax1.set_title('thick edges')

fig.tight_layout()
plt.show()

# Test Data

In [4]:
import numpy as np
from matplotlib.image import NonUniformImage
import matplotlib.pyplot as plt

# Time array
T = 100
dt = 1.0
t = np.arange(0, T, dt)
nt = t.size

# Energy array (log spaced)
E0 = 10.0
E1 = 20000
nE = 32
E = np.logspace(np.log(E0), np.log(E1), nE)

# Differential flux
F = np.random.normal(size=(nE, nt))

# Attempt to plot with NonUniformImage
fig, axes = plt.subplots(figsize=(8,4), nrows=1, ncols=1, squeeze=False)
ax = axes[0,0]
im = NonUniformImage(ax, interpolation='nearest', extent=(t_sec[0], t_sec[-1], E[0], E[-1]),
                     cmap=cm.Purples)
im.set_data(t, E, F)

ax.images.append(im)
ax.set_title('ESpec')

plt.show()

NameError: name 't_sec' is not defined